<a href="https://colab.research.google.com/github/Joaomachado69/AI-PMM-ITBI/blob/joao_machado/CRPA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [37]:
import lightgbm as lgb
from sklearn.model_selection import cross_val_score
import pandas as pd 
import numpy as np
import math
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.metrics import mean_squared_error


In [12]:
df_CRPA = pd.read_csv("zip-code-breakdowns-1.csv")
df_CRPA.head()

,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,...,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
0,10001,44,22,0.50,22,0.50,0,0,44,100,...,44,100,20,0.45,24,0.55,0,0,44,100
1,10002,35,19,0.54,16,0.46,0,0,35,100,...,35,100,2,0.06,33,0.94,0,0,35,100
2,10003,1,1,1.00,0,0.00,0,0,1,100,...,1,100,0,0.00,1,1.00,0,0,1,100
3,10004,0,0,0.00,0,0.00,0,0,0,0,...,0,0,0,0.00,0,0.00,0,0,0,0
4,10005,2,2,1.00,0,0.00,0,0,2,100,...,2,100,0,0.00,2,1.00,0,0,2,100


In [17]:
metrics.SCORERS.keys()

dict_keys(['explained_variance', 'r2', 'max_error', 'neg_median_absolute_error', 'neg_mean_absolute_error', 'neg_mean_absolute_percentage_error', 'neg_mean_squared_error', 'neg_mean_squared_log_error', 'neg_root_mean_squared_error', 'neg_mean_poisson_deviance', 'neg_mean_gamma_deviance', 'accuracy', 'top_k_accuracy', 'roc_auc', 'roc_auc_ovr', 'roc_auc_ovo', 'roc_auc_ovr_weighted', 'roc_auc_ovo_weighted', 'balanced_accuracy', 'average_precision', 'neg_log_loss', 'neg_brier_score', 'adjusted_rand_score', 'rand_score', 'homogeneity_score', 'completeness_score', 'v_measure_score', 'mutual_info_score', 'adjusted_mutual_info_score', 'normalized_mutual_info_score', 'fowlkes_mallows_score', 'precision', 'precision_macro', 'precision_micro', 'precision_samples', 'precision_weighted', 'recall', 'recall_macro', 'recall_micro', 'recall_samples', 'recall_weighted', 'f1', 'f1_macro', 'f1_micro', 'f1_samples', 'f1_weighted', 'jaccard', 'jaccard_macro', 'jaccard_micro', 'jaccard_samples', 'jaccard_wei

In [19]:
df_CRPA.describe()

,JURISDICTION NAME,COUNT PARTICIPANTS,COUNT FEMALE,PERCENT FEMALE,COUNT MALE,PERCENT MALE,COUNT GENDER UNKNOWN,PERCENT GENDER UNKNOWN,COUNT GENDER TOTAL,PERCENT GENDER TOTAL,...,COUNT CITIZEN STATUS TOTAL,PERCENT CITIZEN STATUS TOTAL,COUNT RECEIVES PUBLIC ASSISTANCE,PERCENT RECEIVES PUBLIC ASSISTANCE,COUNT NRECEIVES PUBLIC ASSISTANCE,PERCENT NRECEIVES PUBLIC ASSISTANCE,COUNT PUBLIC ASSISTANCE UNKNOWN,PERCENT PUBLIC ASSISTANCE UNKNOWN,COUNT PUBLIC ASSISTANCE TOTAL,PERCENT PUBLIC ASSISTANCE TOTAL
count,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.0,236.0,236.000000,236.000000,...,236.000000,236.000000,236.000000,236.000000,236.000000,236.000000,236.0,236.0,236.000000,236.000000
mean,11127.173729,17.661017,10.296610,0.243983,7.364407,0.201017,0.0,0.0,17.661017,44.491525,...,17.661017,44.487288,5.974576,0.139195,11.686441,0.305805,0.0,0.0,17.661017,44.491525
std,1052.431790,43.279737,28.189121,0.333677,18.880888,0.295731,0.0,0.0,43.279737,49.801264,...,43.279737,49.796563,16.803729,0.227979,29.369458,0.380602,0.0,0.0,43.279737,49.801264
min,10001.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
25%,10451.750000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
50%,11216.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.000000
75%,11422.250000,13.000000,6.000000,0.515000,4.000000,0.400000,0.0,0.0,13.000000,100.000000,...,13.000000,100.000000,3.250000,0.252500,8.000000,0.662500,0.0,0.0,13.000000,100.000000
max,20459.000000,272.000000,194.000000,1.000000,157.000000,1.000000,0.0,0.0,272.000000,100.000000,...,272.000000,100.000000,155.000000,1.000000,206.000000,1.000000,0.0,0.0,272.000000,100.000000


In [22]:
a_CRPA = df_CRPA.drop(["COUNT RECEIVES PUBLIC ASSISTANCE"],axis=1)
b_CRPA = df_CRPA["COUNT RECEIVES PUBLIC ASSISTANCE"]

In [23]:
a_CRPA_train, a_CRPA_test, b_CRPA_train, b_CRPA_test = train_test_split(a_CRPA,b_CRPA, random_state=42)

In [29]:
regressor_lgbm = lgb.LGBMRegressor()

cross_val_score(regressor_lgbm,a_CRPA_train,b_CRPA_train,scoring="neg_root_mean_squared_error").mean()

-10.461582988609774

In [30]:
regressor_lgbm_max_depth = lgb.LGBMRegressor(max_depth=2)

cross_val_score(regressor_lgbm_max_depth,a_CRPA_train,b_CRPA_train,scoring="neg_root_mean_squared_error").mean()

-10.282484059515738

In [31]:
regressor_lgbm_dart = lgb.LGBMRegressor(boosting_type="dart")
cross_val_score(regressor_lgbm_dart,a_CRPA_train,b_CRPA_train,scoring="neg_root_mean_squared_error").mean()

-10.95316426554756

In [32]:
regressor_final = regressor_lgbm_max_depth

In [33]:
regressor_final.fit(a_CRPA_train,b_CRPA_train, eval_metric="")

LGBMRegressor(max_depth=2)

In [35]:
predicoes_CRPA = regressor_final.predict(a_CRPA_test)
predicoes_CRPA[:10]

array([ 8.54279237,  0.51457235, -0.68572807,  1.42540969,  3.68069301,
       -0.27602544, 48.1982388 , -0.27602544,  0.20305194,  3.19703517])

In [36]:
b_CRPA_test[:10]

69      3
190     0
181     0
9       0
127     4
109     0
229    77
93      0
150     0
15      0
Name: COUNT RECEIVES PUBLIC ASSISTANCE, dtype: int64

In [38]:
mse = mean_squared_error(b_CRPA_test, predicoes_CRPA)
display(mse)

rmse = math.sqrt(mse)
rmse

72.3909076456381

8.508284647661837